In [8]:
%%time

from functools import partial
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 79.9 µs


In [9]:
detector = FastAIScreenDetector()
method_params = list(detector.methods.keys())
train_params = list(detector.train_params.keys())
all_params = train_params + method_params

# MODEL PARAMETERS

batch_size = [8]
resize_factor = [2]
frozen_epochs = [6]
unfrozen_epochs = [7]
frozen_lr = [1e-3]
unfrozen_lr = [slice(1e-4, 2e-4)]

train_params_values = [batch_size] + [resize_factor] + [frozen_epochs] + [unfrozen_epochs] + [frozen_lr] + [unfrozen_lr]

# POST PROCESSING PARAMETERS
base = [True]
base_lower_bounds = [5, 7, 10]
base_upper_bounds = [40, 50, 60]
base_factors = [[0.1, 0.01]]

erode_dilate = [True]
erode_dilate_lower_bounds = [5]
erode_dilate_upper_bounds = [40]
erode_dilate_factors = [[0.1, 0.01]]
erode_dilate_iterations = [40, 100]

ratio_split = [True]
ratio_split_lower_bounds = [0.7, 0.9]
ratio_split_upper_bounds = [1.5]

methods_values = [base] + [erode_dilate] + [ratio_split] + [base_lower_bounds] + [base_upper_bounds] \
        + [base_factors] + [erode_dilate_lower_bounds] + [erode_dilate_upper_bounds] + [erode_dilate_factors] \
        + [erode_dilate_iterations] + [ratio_split_lower_bounds] + [ratio_split_upper_bounds]

In [10]:
method_settings = list(product(*methods_values))
train_settings = list(product(*train_params_values))
all_lectures = [video.filename for video in ALL_VIDEOS]
all_frames = [frame for video in ALL_VIDEOS for frame in video]
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}
test_lectures = ['PB069-D2-20140305.mp4']
test_frames = [frame for lecture in test_lectures for frame in all_frames_grouped_by_videos[lecture]]
actual_detector = AnnotatedSampledVideoScreenDetector()

In [11]:
Split = namedtuple('Split', ['train', 'valid'])
kf = KFold(n_splits=5, shuffle=True, random_state=123)
splits = {}
for j, split in enumerate(kf.split(all_lectures)):    
    train_lectures = [all_lectures[index] for index in split[0]]
    valid_lectures = [all_lectures[index] for index in split[1]]
    valid_frames = [frame for lecture in valid_lectures for frame in all_frames_grouped_by_videos[lecture]]
    splits[j] = Split(train=train_lectures, valid=valid_lectures)

In [12]:
j = 0

In [ ]:
# Model selection
method_settings = list(product(*methods_values))
train_settings = list(product(*train_params_values))
df_all = pd.DataFrame(columns=all_params + ['iou', 'wrong_count', 'kfold_split'])
split = splits[j]

# train_setting = train_settings[0]
for train_setting in tqdm(train_settings):
    train_params_dict = dict(zip(train_params, train_setting))    
    train_lectures = split.train
    valid_lectures = split.valid
    valid_frames = [frame for lecture in valid_lectures for frame in all_frames_grouped_by_videos[lecture]]

    filtered_by = lambda name: any([lecture in str(name) for lecture in train_lectures + valid_lectures])  \
                    and 'frame' in str(name)
    split_by = lambda name: any([lecture in str(name) for lecture in valid_lectures])

    detector = FastAIScreenDetector(train_params=train_params_dict, methods=None, filtered_by=filtered_by,
                                valid_func=split_by, device='cuda')

    detector.train()

    actuals = [actual_detector.detect(frame) for frame in valid_frames]
    sem_preds = detector.semantic_segmentation_batch(valid_frames)

    print(f"Iterating through {len(method_settings)} methods in split {j}.")    
    for i, method_setting in enumerate(method_settings):    
        preds = detector.post_processing_batch(sem_preds, valid_frames, dict(zip(method_params, method_setting)))
        wrong_count, ious, _ = evaluate(actuals, preds)
        
        iou_score = np.nanmean(ious)
        wrong_count = len(wrong_count)
    df_all.loc[len(df_all)] = train_setting + method_setting + (iou_score, wrong_count, j)
df_all.to_csv(f"/content/drive/My Drive/split_{j}.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

epoch,train_loss,valid_loss,acc,dice,iou,time


In [0]:
# %%time
# # Model selection
# method_settings = list(product(*methods_values))
# train_settings = list(product(*train_params_values))
# df_all = pd.DataFrame(columns=all_params + ['iou', 'wrong_count', 'kfold_split'])

# for train_setting in tqdm(train_settings):
#     train_params_dict = dict(zip(train_params, train_setting))
#     for i in splits.keys():
#         train_lectures = splits[i].train
#         valid_lectures = splits[i].valid
#         valid_frames = [frame for lecture in valid_lectures for frame in all_frames_grouped_by_videos[lecture]]

#         filtered_by = lambda name: any([lecture in str(name) for lecture in train_lectures + valid_lectures])  \
#                         and 'frame' in str(name)
#         split_by = lambda name: any([lecture in str(name) for lecture in valid_lectures])
        
#         detector = FastAIScreenDetector(train_params=train_params_dict, methods=None, filtered_by=filtered_by,
#                                     valid_func=split_by, device='cuda')
    
#         detector.train()
        
#         actuals = [actual_detector.detect(frame) for frame in valid_frames]
#         sem_preds = detector.semantic_segmentation_batch(valid_frames)
        
#         print(f"Iterating through {len(method_settings)} methods in split {j}.")    
#         for i, method_setting in enumerate(method_settings):    
#             preds = detector.post_processing_batch(sem_preds, valid_frames, dict(zip(method_params, method_setting)))
#             wrong_count, ious, _ = evaluate(actuals, preds)
            
#             iou_score = np.nanmean(ious)
#             wrong_count = len(wrong_count)
#             df_all.loc[len(df_all)] = train_setting + method_setting + (iou_score, wrong_count, j)

  0%|          | 0/1 [00:00<?, ?it/s]Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


epoch,train_loss,valid_loss,acc,dice,iou,time


RuntimeError: ignored

In [0]:
df_all.to_csv("/content/drive/My Drive/model_selection2.csv")

In [0]:
df_all = pd.read_csv('/content/drive/My Drive/model_selection2.csv')

In [0]:
unhashable_columns = ['frozen_lr', 'unfrozen_lr', 'base_factors', 'erode_dilate_factors']
df_all[unhashable_columns] = df_all[unhashable_columns].astype(str)
df_all['wrong_count'] = df_all['wrong_count'].astype(int)

In [0]:
best_params = df_all.groupby(train_params + method_params).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name
converted_params = []
for i, par in enumerate(best_params):
    if isinstance(par, np.int64) or isinstance(par, np.float64):
        converted_params.append(par.item())
    else:
        converted_params.append(par)
best_params = tuple(converted_params)
print(df_all.groupby(train_params + method_params).mean().sort_values(by=['wrong_count', 'iou']).iloc[0])

Unnamed: 0     28.000000
iou             0.778530
wrong_count    35.333333
kfold_split     1.000000
Name: (8, 2, 7, 6, 0.001, slice(0.0001, 0.0002, None), True, True, True, 5, 50, [0.1, 0.01], 5, 40, [0.1, 0.01], 40, 0.7, 1.5), dtype: float64


In [0]:
best_methods = dict(zip(method_params, best_params[-len(method_params):]))
best_methods['base_factors'] = eval(best_methods['base_factors'])
best_methods['erode_dilate_factors'] = eval(best_methods['erode_dilate_factors'])
best_train_params_dict = dict(zip(train_params, best_params[:len(train_params)]))
best_train_params_dict['frozen_lr'] = float(best_train_params_dict['frozen_lr'])
best_train_params_dict['unfrozen_lr'] = eval(best_train_params_dict['unfrozen_lr'])
filtered_by = lambda name: 'frame' in str(name)
split_by = lambda name: any([lecture in str(name) for lecture in test_lectures])

best_detector = FastAIScreenDetector(train_params=best_train_params_dict, methods=best_methods, 
                                        filtered_by=filtered_by, valid_func=split_by, progressbar=True, device='cuda')

# best_detector.train()

In [0]:
best_detector.save()

In [0]:
actuals = [actual_detector.detect(frame) for frame in test_frames]
preds = [best_detector.detect(frame) for frame in test_frames]
wrong_count, iou, _ = evaluate(actuals, preds)
iou_score = np.nanmean(ious)
wrong_count = len(wrong_count)

In [0]:
iou_score, wrong_count

In [0]:
def single_frame_visualization(frame, actual_screens, pred_screens):
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 4), sharex='row', sharey='row')
    fig.tight_layout()
    for screen in actual_screens:
        polygon = screen.coordinates._polygon
        axes[0].set_title("Actual")
        axes[0].plot(*polygon.exterior.xy, c='tab:orange')
        axes[2].plot(*polygon.exterior.xy, c='tab:orange', label="Actual")

    for screen in pred_screens:
        polygon = screen.coordinates._polygon
        axes[1].set_title("Prediction")
        axes[1].plot(*polygon.exterior.xy, c='tab:blue')
        axes[2].plot(*polygon.exterior.xy, c='tab:blue', label="Predicted")

    axes[2].set_title("Combined")
    legend_without_duplicate_labels(axes[2])

    axes[3].set_title("Original")
    axes[3].imshow(frame.image)
    plt.show()


In [0]:
semantic_segmentations = best_detector.semantic_segmentation_batch(all_frames[:20])

In [0]:
for frame in all_frames[:20]:
    plt.imshow(frame.image)
    plt.show()

In [0]:
for segmentation in semantic_segmentations:
    plt.imshow(segmentation)
    plt.show()

NameError: ignored

In [0]:
for i, _ in enumerate(test_frames):
    single_frame_visualization(test_frames[i], actuals[i], preds[i])